In [ ]:
import os
import re
import sys
import glob
import pickle
import tables
from tqdm import tqdm
from scipy.fft import fft, fftfreq
from scipy.signal import butter, filtfilt, hilbert

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
# %matplotlib notebook

import tensorflow as tf
from tensorflow import keras

if '..' not in sys.path:
    sys.path.append('..')
from dlml.utils import collect_experiments
from dlml.data import read_area_values, load_data_areas, load_data_slide
from dlml.nn import predict, DownSampling1D, SpectralPooling, MaxPooling1DWithArgmax, compute_receptive_field

#### Find the best experiment given a set of tags

In [ ]:
area_ID = 1
area_measure = 'momentum'
stoch_load_bus_IDs = []
rec_bus_IDs = [3]
H_G1, D, DZA = None, None, None # 500, 2, 0
additional_tags = ['ReLU_none', 'converted_from_PowerFactory', 'all_stoch_loads', 'data_subset']
missing_tags = []
use_FFT = False
if use_FFT:
    additional_tags.append('fft')
else:
    missing_tags.append('fft')
pooling_type = ''
if pooling_type is not None and pooling_type != '':
    additional_tags.append(pooling_type + '_pooling')

# training on frequency data, 2 output values
# experiment_ID = '9ea493c789b542bf979c51a6031f4044'
# training on frequency data, 4 output values
# experiment_ID = 'f6d9a03f1cfe450288e9cb86da94235f'
# training on time series data, 2 output values
experiment_ID = '034a1edb0797475b985f0e1335dab383'
# training on time series data, 4 output values
# experiment_ID = 'b346a89d384c4db2ba4058a2c83c4f12'
# training on time series data, 2 output values, with MaxPooling1DWithArgmax layer
# experiment_ID = '9034f8bc4f874c938dfa5f1f9ee04e82'
# experiment_ID = None

if experiment_ID is not None:
    from comet_ml.api import API, APIExperiment
    api = API(api_key = os.environ['COMET_API_KEY'])
    experiment = api.get_experiment('danielelinaro', 'inertia', experiment_ID)
    sys.stdout.write(f'Getting metrics for experiment {experiment_ID[:6]}... ')
    sys.stdout.flush()
    metrics = experiment.get_metrics()
    sys.stdout.write('done.\n')
    val_loss = []
    for m in metrics:
        if m['metricName'] == 'val_loss':
            val_loss.append(float(m['metricValue']))
        elif m['metricName'] == 'mape_prediction':
            MAPE = float(m['metricValue'])
    val_loss = np.array(val_loss)
else:
    # find the best experiment that matches the set of tags above
    experiments = collect_experiments(area_ID, area_measure=area_measure, D=D, DZA=DZA, \
                                      stoch_load_bus_IDs=stoch_load_bus_IDs, H_G1=H_G1, \
                                      rec_bus_IDs=rec_bus_IDs, additional_tags=additional_tags, \
                                      missing_tags=missing_tags, verbose=True)
    experiment_IDs = list(experiments.keys())
    experiment_ID = experiment_IDs[np.argmin([expt['val_loss'].min() for expt in experiments.values()])]
#     experiment_ID = experiment_IDs[np.argmin([expt['MAPE'] for expt in experiments.values()])]

    MAPE = experiments[experiment_ID]['MAPE']
    loss = experiments[experiment_ID]['loss']
    val_loss = experiments[experiment_ID]['val_loss']
    batch_loss = experiments[experiment_ID]['batch_loss']
    tags = experiments[experiment_ID]['tags']
print(f'Selected experiment is {experiment_ID[:6]} (val_loss = {val_loss.min():.4f}, MAPE = {MAPE:.4f}%).')

#### Load the model

In [ ]:
experiments_path = '../experiments/neural_network/'
network_parameters = pickle.load(open(os.path.join(experiments_path, experiment_ID, 'parameters.pkl'), 'rb'))
checkpoint_path = experiments_path + experiment_ID + '/checkpoints/'
checkpoint_files = glob.glob(checkpoint_path + '*.h5')
try:
    epochs = [int(os.path.split(file)[-1].split('.')[1].split('-')[0]) for file in checkpoint_files]
    best_checkpoint = checkpoint_files[epochs.index(np.argmin(val_loss) + 1)]
except:
    best_checkpoint = checkpoint_files[-1]
try:
    model = keras.models.load_model(best_checkpoint)
except:
    if pooling_type == 'downsample':
        custom_objects = {'DownSampling1D': DownSampling1D}
    elif pooling_type == 'spectral':
        custom_objects = {'SpectralPooling': SpectralPooling}
    elif pooling_type == 'argmax':
        custom_objects = {'MaxPooling1DWithArgmax': MaxPooling1DWithArgmax}
    with keras.utils.custom_object_scope(custom_objects):
        model = keras.models.load_model(best_checkpoint)
if pooling_type == 'argmax':
    for layer in model.layers:
        if isinstance(layer, MaxPooling1DWithArgmax):
            print(f'Setting store_argmax = True for layer "{layer.name}".')
            layer.store_argmax = True
x_train_mean = network_parameters['x_train_mean']
x_train_std  = network_parameters['x_train_std']
try:
    x_train_min = network_parameters['x_train_min']
    x_train_max = network_parameters['x_train_max']
except:
    pass
var_names = network_parameters['var_names']
print(f'Loaded network from {best_checkpoint}.')
print(f'Variable names: {var_names}')

#### Plot the model topology

In [ ]:
model.summary()

### Effective receptive field size and stride of the model

In [ ]:
effective_RF_size,effective_stride = compute_receptive_field(model, stop_layer=keras.layers.Flatten)
print('Effective receptive field size:')
for i,(k,v) in enumerate(effective_RF_size.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))
print()
print('Effective stride:')
for i,(k,v) in enumerate(effective_stride.items()):
    print(f'{i}. {k} ' + '.' * (20 - len(k)) + ' {:d}'.format(v))

#### Load the data set

In [ ]:
set_name = 'test'
use_fft = network_parameters['use_fft'] if 'use_fft' in network_parameters else False
data_dirs = []
for area_ID,data_dir in zip(network_parameters['area_IDs'], network_parameters['data_dirs']):
    data_dirs.append(data_dir.format(area_ID))
data_dir = os.path.join('..', data_dirs[0])
data_files = sorted(glob.glob(data_dir + os.path.sep + f'*_{set_name}_set.h5'))
ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=use_fft)
y = ret[2][set_name]
if use_fft:
    X = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min, x_train_max))]
    F = ret[0]
else:
    X = [(ret[1][set_name][i] - m) / s for i,(m,s) in enumerate(zip(x_train_mean, x_train_std))]
    t = ret[0]

#### Predict the momentum using the model

In [ ]:
idx = [np.where(y == mom)[0] for mom in np.unique(y)]
n_mom_values = len(idx)
momentum = [np.squeeze(model.predict(X[0][jdx])) for jdx in idx]
mean_momentum = [m.mean() for m in momentum]
stddev_momentum = [m.std() for m in momentum]
print('Mean momentum:', mean_momentum)
print(' Std momentum:', stddev_momentum)

### Plot the inputs and their FFTs

In [ ]:
cmap = plt.get_cmap('tab10', n_mom_values)
if use_fft:
    Xf = X
else:
    data_files_training = sorted(glob.glob(data_dir + os.path.sep + f'*_training_set.h5'))
    if len(data_files_training) == 0:
        data_files_training = sorted(glob.glob(data_dir + os.path.sep + f'*_test_set.h5'))
    ret_fft = load_data_areas({'training': data_files_training}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=200,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    x_train_min_fft = np.array([val.min() for val in ret_fft[1]['training']], dtype=np.float32)
    x_train_max_fft = np.array([val.max() for val in ret_fft[1]['training']], dtype=np.float32)
    ret = load_data_areas({set_name: data_files}, network_parameters['var_names'],
                        network_parameters['generators_areas_map'][:1],
                        network_parameters['generators_Pnom'],
                        network_parameters['area_measure'],
                        trial_dur=network_parameters['trial_duration'],
                        max_block_size=100,
                        use_tf=False, add_omega_ref=True,
                        use_fft=True)
    F = ret[0]
    Xf = [(ret[1][set_name][i] - m) / (M - m) for i,(m,M) in enumerate(zip(x_train_min_fft,
                                                                           x_train_max_fft))]

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(10, 4))

for j,jdx in enumerate(idx):
    mean = X[0][jdx].mean(axis=0)
    stddev = X[0][jdx].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(jdx.size)
    ax[0].fill_between(t, mean + ci, mean - ci, color=cmap(j))
    mean = Xf[0][jdx].mean(axis=0)
    stddev = Xf[0][jdx].std(axis=0)
    ci = 1.96 * stddev / np.sqrt(jdx.size)
    ax[1].fill_between(F, mean + ci, mean - ci, color=cmap(j))

for a in ax:
    for side in 'right','top':
        a.spines[side].set_visible(False)
    a.grid(which='major', axis='both', ls=':', lw=0.5, color=[.6,.6,.6])
ax[1].set_xscale('log')
ax[0].set_xlabel('Time [min]')
ax[0].set_ylabel('Normalized trace')
ax[1].set_xlabel('Frequency [Hz]')
ax[1].set_ylabel('Normalized FFT')
fig.tight_layout()
fig.savefig(f'spectra_{n_mom_values}_momentum_levels.pdf')

### Build a model with as many outputs as there are convolutional or pooling layers

In [ ]:
outputs = [layer.output for layer in model.layers \
           if layer.name in effective_RF_size.keys() and not isinstance(layer, keras.layers.InputLayer)]
multi_output_model = keras.Model(inputs=model.inputs, outputs=outputs)
# Y = [multi_output_model.predict(X[0][jdx]) for jdx in idx]
print(f'The model has {len(outputs)} outputs, corresponding to the following layers:')
for i,layer in enumerate(multi_output_model.layers):
    if not isinstance(layer, keras.layers.InputLayer):
        print(f'    {i}. {layer.name}')

In [ ]:
def compute_correlation(model, x, dt, bands,
                        effective_RF_size, effective_stride,
                        filter_order=6, verbose=False):

    ## Filter the input in a series of bands and compute the signal envelope
    N_samples = x.size
    N_bands = len(bands)
    # filter the input in various frequency bands
    x_filt = np.zeros((N_bands, N_samples))
    for i in range(N_bands):
        b,a = butter(filter_order//2, bands[i], 'bandpass', fs=1/dt)
        x_filt[i,:] = filtfilt(b, a, x)
    # compute the envelope of the filtered signal
    x_filt_envel = np.abs(hilbert(x_filt))
    
    ## Compute the outputs of the last layer before the fully connected layer
    layer_name = model.layers[-1].name
    multi_y = model(x[np.newaxis, :])
    y = np.squeeze(multi_y[-1].numpy())
    N_neurons, N_filters = y.shape
    if verbose:
        print(f'Layer "{layer_name}" has {N_filters} filters, each with {N_neurons} neurons.')
    
    ## Compute the mean squared envelope for each receptive field
    RF_sz, RF_str = effective_RF_size[layer_name], effective_stride[layer_name]
    if verbose:
        print(f'The effective RF size and stride of layer "{layer_name}" are {RF_sz} and {RF_str} respectively.')
    squared_mean_envel = np.zeros((N_bands, N_neurons, N_filters))
    mean_squared_envel = np.zeros((N_bands, N_neurons, N_filters))
    for i in range(N_neurons):
        start = i * RF_str
        stop = start + RF_sz
        for j in range(N_filters):
            x_filt_envel_sub = x_filt_envel[:, start : stop]
            squared_mean_envel[:, i, j] = np.mean(x_filt_envel_sub, axis=1) ** 2
            mean_squared_envel[:, i, j] = np.mean(x_filt_envel_sub ** 2, axis=1)
            
    ## For each frequency band, compute the correlation between mean squared envelope
    ## of the input (to each receptive field) and the output of each neuron in the layer
    R = np.zeros((N_bands, N_filters))
    for i in range(N_bands):
        for j in range(N_filters):
            R[i, j] = np.corrcoef(y[:, j], mean_squared_envel[i, :, j])[0,1]
            
    return R

In [ ]:
dt = np.diff(t[:2])[0]
edges = np.logspace(-1, 1, 20)
bands = [[a,b] for a,b in zip(edges[:-1], edges[1:])]
N_bands = len(bands)
N_filters = multi_output_model.layers[-1].output.shape[-1]
N_traces = 1000
R = np.zeros((N_traces, N_bands, N_filters))
for i in tqdm(range(N_traces)):
    R[i,:,:] = compute_correlation(multi_output_model, X[0][i,:], dt, bands,
                                   effective_RF_size, effective_stride)

1. Choose the frequency bands appropriately
1. Perform the same type of analysis on an untrained network

In [ ]:
R_mean = R.mean(axis=0)
edge = 10
idx = np.argsort(R_mean[edge,:])
R_mean = R_mean[:,idx]

cmap = plt.get_cmap('bwr')
fig,ax = plt.subplots(1, 1, figsize=(8, 5))
x = np.arange(N_filters)
y = edges[:-1]
X,Y = np.meshgrid(x, y)
im = ax.pcolor(X, Y, R_mean, shading='auto', cmap=cmap)
ax.set_yscale('log')
cbar = plt.colorbar(im, label='Correlation')
ax.set_xlabel('Filter #')
ax.set_ylabel('Frequency [Hz]')
for side in 'right','top':
    ax.spines[side].set_visible(False)
fig.tight_layout()

### Filter the input in a series of bands and compute the signal envelope

In [ ]:
# take an input time series
i = 0
x = X[0][i,:]
# filter it in various frequency ranges
dt = np.diff(t[:2])[0]
filter_order = 6
bands = [[0.1, 0.2], [0.5, 2], [3, 10]]
N_bands = len(bands)
x_filt = []
for band in bands:
    b,a = butter(filter_order//2, band, 'bandpass', fs=1/dt)
    x_filt.append(filtfilt(b, a, x))
x_filt = np.array(x_filt)
# compute the envelope of the filtered signal
x_filt_envel = np.abs(hilbert(x_filt))

In [ ]:
fig,ax = plt.subplots(1, 1, figsize=(10,6))
ax.plot(t, x, 'k', lw=1, label='Broad-band')
cmap = plt.get_cmap('spring', n_bands)
for i in range(n_bands):
    ax.plot(t, x_filt[i,:], color=cmap(i), lw=1, label=bands[i])
    ax.plot(t, x_filt_envel[i,:], color=cmap(i), lw=3)
for side in 'right','top':
    ax.spines[side].set_visible(False)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Normalized voltage')
ax.legend(loc='best', frameon=False)
fig.tight_layout()

### Compute the outputs of the last layer before the fully connected layers

In [ ]:
multi_y = multi_output_model(x[np.newaxis, :])
y = np.squeeze(multi_y[-1].numpy())
N_neurons, N_filters = y.shape
print(f'Layer "{layer_name}" has {N_filters} filters, each with {N_neurons} neurons.')

### Compute the mean squared envelope for each receptive field

In [ ]:
layer_id = 6
layer_name = model.layers[layer_id].name
RF_sz, RF_str = effective_RF_size[layer_name], effective_stride[layer_name]
print(f'The effective RF size and stride of layer "{layer_name}" are {RF_sz} and {RF_str} respectively.')

In [ ]:
squared_mean_envel = np.zeros((N_bands, N_neurons, N_filters))
mean_squared_envel = np.zeros((N_bands, N_neurons, N_filters))
for i in range(N_neurons):
    start = i * RF_str
    stop = start + RF_sz
    for j in range(N_filters):
        x_filt_envel_sub = x_filt_envel[:, start : stop]
        squared_mean_envel[:, i, j] = np.mean(x_filt_envel_sub, axis=1) ** 2
        mean_squared_envel[:, i, j] = np.mean(x_filt_envel_sub ** 2, axis=1)

In [ ]:
R = np.zeros((N_bands, N_filters))
for i in range(N_bands):
    for j in range(N_filters):
        R[i, j] = np.corrcoef(y[:, j], mean_squared_envel[i, :, j])[0,1]